In [1]:
import pickle
from random import randrange
import itertools
import random
import pandas as pd
import numpy as np
import math
from gams import *

cs = 3                         #number of candidate charging station

trip_information_3line = pd.read_csv("Trip_information_3line.csv")
terminal_distance_3line = pd.read_csv("Between_terminal_distance_3line.csv")

######function to calculate the travel time and energy needed between each trip and the depot
def depot_to_trip(trip_information, terminal_distance):
    from_depot_to_trip_time = []
    from_depot_to_trip_energy = []
    from_trip_to_depot_time = []
    from_trip_to_depot_energy = []
    i=0
    while i < 70:
        trip = trip_information.iloc[i]["Trip_no"]
        trip_starttime = trip_information.iloc[i]["Start_time"]
        trip_endtime = trip_information.iloc[i]["End_time"]
        trip_startterminal = trip_information.iloc[i]["Start_terminal"]
        trip_endterminal = trip_information.iloc[i]["End_terminal"]
        from_depot_distance = terminal_distance.iloc[int(trip_startterminal)-1]["Depot"]
        to_depot_distance = terminal_distance.iloc[int(trip_endterminal)-1]["Depot"]
        depot_to_trip_time_need = from_depot_distance/20
        trip_to_depot_time_need = to_depot_distance/20
        depot_to_trip_energy_need = from_depot_distance*2.69
        trip_to_depot_energy_need = to_depot_distance*2.69
        from_depot_to_trip_time.append(depot_to_trip_time_need)
        from_trip_to_depot_time.append(trip_to_depot_time_need)
        from_depot_to_trip_energy.append(depot_to_trip_energy_need)
        from_trip_to_depot_energy.append(trip_to_depot_energy_need)
        i += 1
    return from_depot_to_trip_time, from_trip_to_depot_time, from_depot_to_trip_energy, from_trip_to_depot_energy

from_depot_to_trip_time, from_trip_to_depot_time, from_depot_to_trip_energy, from_trip_to_depot_energy = depot_to_trip(trip_information_3line, terminal_distance_3line)

from_depot_to_trip_time_3line = pd.DataFrame(from_depot_to_trip_time) 
from_trip_to_depot_time_3line = pd.DataFrame(from_trip_to_depot_time)
from_depot_to_trip_energy_3line = pd.DataFrame(from_depot_to_trip_energy)
from_trip_to_depot_energy_3line = pd.DataFrame(from_trip_to_depot_energy)
from_depot_to_trip_time_3line.to_csv("From_depot_to_trip_time_3line")
from_trip_to_depot_time_3line.to_csv("From_trip_to_depot_time_3line")
from_depot_to_trip_energy_3line.to_csv("From_depot_to_trip_energy_3line")
from_trip_to_depot_energy_3line.to_csv("From_trip_to_depot_energy_3line")

In [2]:
From_depot_to_trip_time = pd.read_csv("From_depot_to_trip_time_3line")
From_trip_to_depot_time = pd.read_csv("From_trip_to_depot_time_3line")
From_depot_to_trip_energy = pd.read_csv("From_depot_to_trip_energy_3line")
From_trip_to_depot_energy = pd.read_csv("From_trip_to_depot_energy_3line")

In [3]:
From_depot_to_trip_energy

,Unnamed: 0,0
0,0,0.538
1,1,0.538
2,2,2.152
3,3,0.538
4,4,0.538
...,...,...
65,65,0.538
66,66,0.538
67,67,0.538
68,68,0.538


In [4]:
terminal_distance_3line

,Unnamed: 0,1,2,3,Depot
0,1,0.00,7.12,5.7,5.7
1,2,7.12,0.00,1.1,0.8
2,3,5.70,1.10,0.0,0.2


In [5]:
######function to get the maximum available time for charging between trips and the energy needed between trips
def generate_between_trip(trip_information, terminal_distance):
    between_trip_time = pd.DataFrame()
    between_trip_energy = pd.DataFrame()
    i=0
    while i < 70:
        time_list = []
        energy_list = []
        j = 0
        while j < 70:
            trip1 = trip_information.iloc[i]["Trip_no"]
            trip2 = trip_information.iloc[j]["Trip_no"]
            trip1_starttime = trip_information.iloc[i]["Start_time"]
            trip2_starttime = trip_information.iloc[j]["Start_time"]
            trip1_endtime = trip_information.iloc[i]["End_time"]
            trip2_endtime = trip_information.iloc[j]["End_time"]
            trip1_startterminal = trip_information.iloc[i]["Start_terminal"]
            trip2_startterminal = trip_information.iloc[j]["Start_terminal"]
            trip1_endterminal = trip_information.iloc[i]["End_terminal"]
            trip2_endterminal = trip_information.iloc[j]["End_terminal"]
            between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
            between_time_need = between_distance/20
            between_energy_need = between_distance*2.69
            if trip1 == trip2:
                time_list.append(-1000)
                energy_list.append(-1000)
            else:
                if trip2_starttime < trip1_endtime + between_time_need*60:
                    time_list.append(-1000)
                    energy_list.append(-1000)
                else:
                    remain_time = (trip2_starttime - trip1_endtime)/60 - between_time_need
                    time_list.append(remain_time)
                    energy_list.append(between_energy_need)
            j += 1
        time_list = pd.DataFrame(time_list)
        energy_list = pd.DataFrame(energy_list)
        time_list = time_list.transpose()
        energy_list = energy_list.transpose()
        between_trip_time = between_trip_time.append(time_list)
        between_trip_energy = between_trip_energy.append(energy_list)
        i+=1
    return between_trip_time, between_trip_energy

In [6]:
result1, result2 = generate_between_trip(trip_information_3line, terminal_distance_3line)
result1.to_csv("Between_trip_time_NumericalStudy_3line.csv") 
result2.to_csv("Between_trip_energy_NumericalStudy_3line.csv")

available_charging_time = pd.read_csv("Between_trip_time_NumericalStudy_3line.csv")
energy_need = pd.read_csv("Between_trip_energy_NumericalStudy_3line.csv")

In [7]:
available_charging_time

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,60,61,62,63,64,65,66,67,68,69
0,0,-1000.0,-1000.0,-1000.0,-1000.0,0.45,0.7,0.828333,0.950000,0.798333,...,11.700000,11.950000,11.698333,12.450000,12.700000,12.950000,13.450000,13.950000,14.450000,14.950000
1,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,0.2,0.328333,0.450000,0.298333,...,11.200000,11.450000,11.198333,11.950000,12.200000,12.450000,12.950000,13.450000,13.950000,14.450000
2,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,0.110667,0.248333,0.666667,...,10.998333,11.248333,11.566667,11.748333,11.998333,12.248333,12.748333,13.248333,13.748333,14.248333
3,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,10.700000,10.950000,10.698333,11.450000,11.700000,11.950000,12.450000,12.950000,13.450000,13.950000
4,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,10.183333,10.433333,10.181667,10.933333,11.183333,11.433333,11.933333,12.433333,12.933333,13.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.383333,0.883333
66,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,0.383333
67,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000
68,0,-1000.0,-1000.0,-1000.0,-1000.0,-1000.00,-1000.0,-1000.000000,-1000.000000,-1000.000000,...,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000,-1000.000000


In [8]:
######generate following set
def generate_following_set(trip_information, terminal_distance):
    following_set_list = []
    i=0
    while i < 70:
        following_set = []
        j = 0
        while j < 70:
            trip1 = trip_information.iloc[i]["Trip_no"]
            trip2 = trip_information.iloc[j]["Trip_no"]
            trip1_starttime = trip_information.iloc[i]["Start_time"]
            trip2_starttime = trip_information.iloc[j]["Start_time"]
            trip1_endtime = trip_information.iloc[i]["End_time"]
            trip2_endtime = trip_information.iloc[j]["End_time"]
            trip1_startterminal = trip_information.iloc[i]["Start_terminal"]
            trip2_startterminal = trip_information.iloc[j]["Start_terminal"]
            trip1_endterminal = trip_information.iloc[i]["End_terminal"]
            trip2_endterminal = trip_information.iloc[j]["End_terminal"]
            between_distance = terminal_distance.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
            between_time_need = between_distance/20
            if trip2_starttime >= trip1_endtime + between_time_need*60:
                following_set.append(int(trip2))
            j +=1
        following_set_list.append(following_set)
        i += 1
    return following_set_list

In [9]:
following_set_list = generate_following_set(trip_information_3line, terminal_distance_3line)
dict_following_set_3line = {}
for i in range(1,71):
    dict_following_set_3line[i] = following_set_list[i-1]
    
with open('Following_set_dict_3line.pickle', 'wb') as handle:
    pickle.dump(dict_following_set_3line, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
following_set = pickle.load(open('Following_set_dict_3line.pickle','rb'))

In [12]:
following_set

{1: [5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70],
 2: [6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70],
 3: [7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,

In [13]:
#####################################
#define a function to get a following trip of a service trip
#####################################
def return_following(i,following_set):
    following_set_i = following_set[i]
    if len(following_set_i) == 0:
        return -1
    else:
        following_set_i = following_set_i[0:3]
        return(random.choice(following_set_i))

In [14]:
#####################################
#define a function to generate a block given a first trip
#####################################
def block_generation(i,block,following_set):
    while True:
        m = return_following(i,following_set)
        if m != -1: 
            block.append(m)
            i = m
        else: 
            #print("block=")
            #print(block)
            return block

In [15]:
####################################
#define a function to get difference of two lists
####################################
def diff(list1, list2):
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1))))

In [16]:
####################################
#define a function to generate the service trip part of a chromosome
####################################
def trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip):
    while True:
        if len(rest_trips) != 0:
            fist_trip = min(rest_trips)                        #get the trip with the earlest starting time in rest of the trips
            new_block = []
            new_block.append(fist_trip)
            new_block = block_generation(fist_trip,new_block,following_set)  #generate a new block
            rest_trips = diff(rest_trips, new_block)           #get rest of the trips
            chromosome = chromosome + new_block                #add the newly generated block to the chromosome
            trip_chains.append(new_block)
            no_bus += 1
            no_trip.append(len(new_block))
            following_set = {key:[ele for ele in following_set[key] if ele not in chromosome] for key in following_set.keys()}
        else:
            #print("All trips have been assigned to blocks")
            #print("Generated trip part of chromosome =")
            #print(trip_chains)
            #print("Fleet size under this trip schedule =")
            #print(no_bus)
            #print("Number of trips for each bus =")
            #print(no_trip)
            
            return trip_chains, no_bus, no_trip

In [17]:
##########################################
#define a function to generate the charging station part of the chromosome
##########################################
def charpart_chromosome_generation():
    Charging_station_part_chromosome = []
    for i in range(cs):
        Charging_station_part_chromosome.append(random.randint(0, 1))
    return Charging_station_part_chromosome

In [18]:
#########################################
#define a function to randomly assign a battery capacity for each bus in the bus fleet
#########################################
def battery_cap_generation(fleet_size):
    bus_type = [1,2]                   #available bus type in the market
    battery_type = [90,380]            #battery capacity of each bus type
    battery_size = [0]*fleet_size      #define a list to store the battery size of each bus in the bus fleet
    battery_lower = [0]*fleet_size     #define the lower bound of battery energy in bus
    battery_upper = [0]*fleet_size     #define the upper bound of battery energy in bus
    for i in range(fleet_size):        #randomly assign a battery capacity for each bus in the bus fleet
        battery_size[i] = random.choice(battery_type)
        battery_lower[i] =  battery_size[i]*0.2
        battery_upper[i] = battery_size[i]*0.9
    return battery_size, battery_lower, battery_upper

In [19]:
########################################
#define the function to check feasibility of the generated chromosome
########################################
def check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, charpart_chromosome):
    i = 0
    while (i < fleet_size):
        #print("i=" +str(i))
        a = trips_per_bus[i]                        #get the number of trips for the bus i
        battery_l = battery_lower[i]                #get the battery capacity lower bound of the bus i
        battery_u = battery_upper[i]                #get the battery capacity upper bound of the bus i
        trip_chain = trip_chains[i]                 #get the trip chain of the bus i
        #print(trip_chain)
        b = trip_chain[0]                           #get the first trip in the trip chain of the bus i
        remain_energy = battery_u - From_depot_to_trip_energy.iloc[b-1][1] - trip_information_3line.iloc[b-1]["Energy_need"]   #get the remaining energy in bus i after finishing its first trip
        if remain_energy < battery_l:
            #print("The depot to first trip part of the trip chain "+str(i)+" is not feasible")
            i = 1000
            return 0
            break
        else:
            if a == 1:
                remain_energy = remain_energy - From_trip_to_depot_energy.iloc[b-1][1]
                if remain_energy < battery_l:
                    #print("The only trip to depot part of the trip chain "+str(i)+" is not feasible")
                    i = 1000
                    return 0
                    break
                #else:
                    #print("Trip chain "+str(i)+" is feasible")
            elif a > 1:
                j = 1
                while (j < a):
                    #print("j= "+str(j))
                    bb = trip_chain[j] 
                    #print("bb= " +str(bb))
                    cc = trip_chain[j-1] 
                    #print("cc= " +str(cc))
                    remain_energy = remain_energy - energy_need.iloc[cc-1][bb]
                    if remain_energy < battery_l:
                        #print("The between trips part "+str(cc)+" to "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                        i = 1000
                        return 0
                        break
                    else:
                        if j < a-1:
                            stt = trip_information_3line.iloc[bb-1]["Start_terminal"]     #get the start terminal of the next trip
                            remain_energy = remain_energy + charpart_chromosome[int(stt)-1]*p_bus_max*available_charging_time.iloc[cc-1][bb]
                            if remain_energy > battery_u:
                                remain_energy = battery_u
                            else: 
                                remain_energy = remain_energy
                            remain_energy = remain_energy - trip_information_3line.iloc[bb-1]["Energy_need"]
                            if remain_energy < battery_l:
                                #print("The between trips part "+str(bb)+" of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                        else:
                            remain_energy = remain_energy - From_trip_to_depot_energy.iloc[bb-1][1]
                            if remain_energy < battery_l:
                                #print("The last trip to depot part of the trip chain "+str(i)+" is not feasible")
                                i = 1000
                                return 0
                                break
                            #else:
                                #print("Trip chain "+str(i)+" is feasible")
                    j += 1
        i += 1
    return 1

In [20]:
######define the maximum charging power of a bus
p_bus_max = 350

In [21]:
### generate the trip part of the chromosome
rest_trips = [i for i in range(1,71)]
chromosome = []
#crosspoint_trip = []
#crosspoint_batt = []
no_bus = 0
no_trip = []
trip_chains = []
trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

In [22]:
#trip_chromosome=trippart_chromosome[0]
trip_chains=trippart_chromosome[0]
#crossover_point=trippart_chromosome[2]
fleet_size=trippart_chromosome[1]
trips_per_bus=trippart_chromosome[2]

### generate the charging station part of the chromosome
charpart_chromosome = charpart_chromosome_generation()
print("Generated charging station part of chromosome =")
print(charpart_chromosome)

### generate the battery capacity for each bus
battery_cap = battery_cap_generation(fleet_size)
battery_size = battery_cap[0]
battery_lower = battery_cap[1]
battery_upper = battery_cap[2]
print(battery_size)
print(battery_lower)
print(battery_upper)

### check feasibility of the generated chromosome
check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, charpart_chromosome)

Generated charging station part of chromosome =
[1, 0, 0]
[380, 380, 90, 380, 90, 90, 90, 380, 380]
[76.0, 76.0, 18.0, 76.0, 18.0, 18.0, 18.0, 76.0, 76.0]
[342.0, 342.0, 81.0, 342.0, 81.0, 81.0, 81.0, 342.0, 342.0]


In [23]:
check

0

In [24]:
### define a empty dictionary to store feasible chromosomes
dict_feasible_chromosome = {}
ii=1      # number of chromosome    
while (ii<=120):
    if check == 0:
        ### generate the trip part of a new chromosome
        rest_trips = [i for i in range(1,71)]
        chromosome = []
        #crosspoint = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

        #trip_chromosome=trippart_chromosome[0]
        trip_chains=trippart_chromosome[0]
        #crossover_point=trippart_chromosome[2]
        fleet_size=trippart_chromosome[1]
        trips_per_bus=trippart_chromosome[2]

        ### generate the charging station part of a new chromosome
        charpart_chromosome = charpart_chromosome_generation()
        #print("Generated charging station part of chromosome =")
        #print(charpart_chromosome)

        ### generate the battery capacity for each bus
        battery_cap = battery_cap_generation(fleet_size)
        battery_size = battery_cap[0]
        battery_lower = battery_cap[1]
        battery_upper = battery_cap[2]
        #print(battery_size)
        #print(battery_lower)
        #print(battery_upper)
    
        ### check feasibility of the generated new chromosome
        check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, charpart_chromosome)
    
    else:
        dict_feasible_chromosome[ii] = {"char_chromosome": charpart_chromosome, "fleet_size": fleet_size, "trip_chromosome": trip_chains, "battery_cap": battery_cap}
        print('hello')
        ii += 1
        
        ### generate the trip part of a new chromosome
        rest_trips = [i for i in range(1,71)]
        chromosome = []
        #crosspoint = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)

        #trip_chromosome=trippart_chromosome[0]
        trip_chains=trippart_chromosome[0]
        #crossover_point=trippart_chromosome[2]
        fleet_size=trippart_chromosome[1]
        trips_per_bus=trippart_chromosome[2]

        ### generate the charging station part of a new chromosome
        charpart_chromosome = charpart_chromosome_generation()
        #print("Generated charging station part of chromosome =")
        #print(charpart_chromosome)

        ### generate the battery capacity for each bus
        battery_cap = battery_cap_generation(fleet_size)
        battery_size = battery_cap[0]
        battery_lower = battery_cap[1]
        battery_upper = battery_cap[2]
        #print(battery_size)
        #print(battery_lower)
        #print(battery_upper)
    
        ### check feasibility of the generated new chromosome
        check = check_feasibility(fleet_size, trips_per_bus, battery_lower, battery_upper, trip_chains, charpart_chromosome)

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


In [25]:
dict_feasible_chromosome

{1: {'char_chromosome': [1, 1, 1],
  'fleet_size': 10,
  'trip_chromosome': [[1, 7, 12, 19, 23, 30, 37, 42, 47, 53, 62, 67, 70],
   [2, 6, 13, 20, 28, 35, 43, 50, 56, 65, 69],
   [3, 10, 17, 25, 33, 41, 48, 57, 64, 68],
   [4, 11, 18, 29, 31, 40, 46, 58, 66],
   [5, 16, 22, 32, 39, 49, 54, 61],
   [8, 24, 34, 44, 55, 63],
   [9, 15, 27, 45, 52, 59],
   [14, 36, 51, 60],
   [21, 38],
   [26]],
  'battery_cap': ([380, 90, 90, 90, 380, 90, 90, 90, 380, 90],
   [76.0, 18.0, 18.0, 18.0, 76.0, 18.0, 18.0, 18.0, 76.0, 18.0],
   [342.0, 81.0, 81.0, 81.0, 342.0, 81.0, 81.0, 81.0, 342.0, 81.0])},
 2: {'char_chromosome': [1, 1, 1],
  'fleet_size': 10,
  'trip_chromosome': [[1, 6, 13, 20, 26, 34, 36, 45, 52, 59, 61, 68],
   [2, 7, 14, 16, 22, 28, 37, 42, 48, 55, 63, 67, 70],
   [3, 10, 18, 25, 31, 40, 49, 53, 62, 69],
   [4, 12, 17, 23, 30, 41, 47, 56, 65],
   [5, 19, 27, 35, 46, 58, 66],
   [8, 21, 32, 43, 54, 60],
   [9, 15, 33, 44, 57, 64],
   [11, 29, 50],
   [24, 38, 51],
   [39]],
  'battery

In [26]:
##########################################
#choose the smaller one from two items
##########################################
def compare_two(num1, num2):
    if num1 >= num2:
        return num2
    else:
        return num1
    
    
##########################################
#function for implementing the single-point crossover
##########################################
def crossover(l1, l2, k):
    l_new = []
    for i in range(0, k):
        l_new.append(l1[i])
    for j in range(k, len(l2)):
        l_new.append(l2[j])
    return l_new
        

##########################################
#crossover between two chromosomes
##########################################
def cross_over(chrom1, chrom2):
    off_spring = {}
    chrom1_char = chrom1["char_chromosome"]
    chrom2_char = chrom2["char_chromosome"]
    chrom1_trip = chrom1["trip_chromosome"]
    chrom2_trip = chrom2["trip_chromosome"]
    chrom1_batt = chrom1["battery_cap"]
    chrom2_batt = chrom2["battery_cap"]
    chrom1_flze = chrom1["fleet_size"]
    chrom2_flze = chrom2["fleet_size"]
    smaller_fleet_size = compare_two(chrom1_flze, chrom2_flze)
    #mm = randrange(1,2)
    #print(mm)
    nn = randrange(1,smaller_fleet_size-1)
    #print("nn=")
    #print(nn)
    off_char = chrom1_char
    off_trip_raw = crossover(chrom1_trip, chrom2_trip, nn)
    off_flze_raw = len(off_trip_raw)
    
    #delete duplicate elements
    off_trip_fp = off_trip_raw[0:nn] #keep the first half part of the off_trip unchanged
    off_trip_sp = off_trip_raw[nn:off_flze_raw] #get the second half part of the off_trip
    off_trip_fp_com = list(itertools.chain.from_iterable(off_trip_fp)) #combine the elements in the first part of off_trip
    off_trip_sp_new = [[ele for ele in sub if ele not in off_trip_fp_com] for sub in off_trip_sp] #delete the elements in the second half part of the off_trip which are duplicated with the elements in the first half part
    off_trip_new = off_trip_fp + off_trip_sp_new #get the new off spring
    off_trip_new = list(filter(None, off_trip_new)) #delete empty trip chains in the new off spring
    
    #add missing elements
    off_trip_new_com = list(itertools.chain.from_iterable(off_trip_new)) #combine the elements in the new off spring
    all_trips = [*range(1,71,1)] #all service trips
    missing_trips = [ele for ele in all_trips if ele not in off_trip_new_com] #get the missing trips
    missing_trips.sort() #sort the missing trips and see it as a new trip chain
    new_trip_chain = missing_trips
    
    #get the trip part of the off spring
    off_trip_new.append(new_trip_chain) #append the new trip chain to the new off spring
    off_trip = off_trip_new
    #print("off_trip =")
    #print(off_trip)
    
    #get the fleet size of the off spring
    off_flze = len(off_trip)
    
    #get the battery size for each bus
    off_batt = battery_cap_generation(off_flze)
    battery_lower = off_batt[1]
    battery_upper = off_batt[2]
    
    #get the number of trips of each bus
    trips_per_bus = []
    for ele in off_trip:
        trips_per_bus.append(len(ele))
    
    #check whether the off spring is feasible
    check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, off_char)
    #print(check)
    
    #store the offspring if it is feasible otherwise generate a new offspring
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(1,71)]
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            off_trip=trippart_chromosome[0]
            off_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            # generate the charging station part of a new chromosome
            off_char = charpart_chromosome_generation()
        
            ### generate the battery capacity for each bus
            off_batt = battery_cap_generation(off_flze)
            battery_size = off_batt[0]
            battery_lower = off_batt[1]
            battery_upper = off_batt[2]
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(off_flze, trips_per_bus, battery_lower, battery_upper, off_trip, off_char)
            #print(check)
            
        else:
            off_spring = {"char_chromosome": off_char, "fleet_size": off_flze, "trip_chromosome": off_trip, "battery_cap": off_batt}
            break
            
    return off_spring

########################################
#mutation
########################################
def mutation(chrom):
    new_chrom = {}
    chrom_char = chrom["char_chromosome"]
    chrom_trip = chrom["trip_chromosome"]
    chrom_batt = chrom["battery_cap"]
    chrom_flze = chrom["fleet_size"]
    
    trips_per_bus = []
    for ele in chrom_trip:
        trips_per_bus.append(len(ele))
    
    battery_lower = chrom_batt[1]
    battery_upper = chrom_batt[2]
    
    n = random. randint(1,2) 
    #print("n=")
    #print(n)
    
    if n == 1:
        jj = random.randrange(len(chrom_char)) 
        #print("jj=")
        #print(jj)
        chrom_char[jj] = int(not chrom_char[jj])
    elif n == 2:
        ii = random.sample(chrom_trip,2) #randomly choose two trip chains in chrom_trip
        #print("ii=")
        #print(ii)
        res_trip_chains = [ele for ele in chrom_trip if ele not in ii] #delete ii from chrom_trip
        #print("res_trip_chains=")
        #print(res_trip_chains)
        res_trip_com = list(itertools.chain.from_iterable(res_trip_chains))
        #print("res_trip_com=")
        #print(res_trip_com)
        ii_com = list(itertools.chain.from_iterable(ii)) #combine the elements in ii
        #print("ii_com")
        #print(ii_com)
        
        chromosome = []
        no_bus = 0
        no_trip = []
        trip_chains = []
        
        following_set_m = {key:[ele for ele in following_set[key] if ele not in res_trip_com] for key in following_set.keys()}
        #print("following_set_m=")
        #print(following_set_m)
        
        new_trip_chains = trippart_chromosome_generation(ii_com, chromosome, trip_chains, following_set_m, no_bus, no_trip)
        new_trip_chains = new_trip_chains[0]
        chrom_trip = res_trip_chains + new_trip_chains
        #print("chrom_trip=")
        #print(chrom_trip)
        chrom_flze = len(chrom_trip)
        #print("chrom_flze=")
        #print(chrom_flze)
        chrom_batt = battery_cap_generation(chrom_flze)
        
        battery_lower = chrom_batt[1]
        battery_upper = chrom_batt[2]
        
        #get the number of trips of each bus
        trips_per_bus = []
        for ele in chrom_trip:
            trips_per_bus.append(len(ele))
    
    #check feasibility of the new chromosome
    check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, chrom_char)
    #print("check=")
    #print(check)
    
    #store the new chromosome if it is feasible otherwise generate a new chromosome
    while True:
        if check != 1:
            # generate the trip part of a new chromosome
            rest_trips = [i for i in range(1,71)]
            chromosome = []
            no_bus = 0
            no_trip = []
            trip_chains = []
            trippart_chromosome = trippart_chromosome_generation(rest_trips, chromosome, trip_chains, following_set, no_bus, no_trip)
        
            chrom_trip=trippart_chromosome[0]
            chrom_flze=trippart_chromosome[1]
            trips_per_bus=trippart_chromosome[2]
        
            # generate the charging station part of a new chromosome
            chrom_char = charpart_chromosome_generation()
        
            ### generate the battery capacity for each bus
            chrom_batt = battery_cap_generation(chrom_flze)
            battery_size = chrom_batt[0]
            battery_lower = chrom_batt[1]
            battery_upper = chrom_batt[2]
        
            ### check feasibility of the generated new chromosome
            check = check_feasibility(chrom_flze, trips_per_bus, battery_lower, battery_upper, chrom_trip, chrom_char)
            #print("check=")
            #print(check)
        
        else:
            new_chrom = {"char_chromosome": chrom_char, "fleet_size": chrom_flze, "trip_chromosome": chrom_trip, "battery_cap": chrom_batt}
            break
            
    return new_chrom     
    
######################################
#genetic algorithm
######################################

###########select
def select_mating_pool(init_population, num_parents):
    fitness_list = []
    fitness_return_list = []
    pop_size = len(init_population)
    for i in range(1,pop_size+1):
        chromosome = init_population[i]
        Charging_station = chromosome["char_chromosome"]
        trip_chains = chromosome["trip_chromosome"]
        chrom_batt = chromosome["battery_cap"]
        fleet_size = chromosome["fleet_size"]
        ########import the parameters of the chromosome to gams
        # store trip chains to a dictionary
        dict_trip_set = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in trip_set:
                dict_trip_set[str(i),str(j)] = 1
        #print(dict_trip_set)

        # store the first trip of each bus to a dictionary
        dict_first_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            dict_first_trip[str(i), str(trip_set[0])] = 1
        #print(dict_first_trip)

        # store the last trip of each bus to a dictionary
        dict_last_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            dict_last_trip[str(i), str(trip_set[-1])] = 1
        #print(dict_last_trip)

        # store the adjacent trips of each bus to a dictionary
        dict_adjacent_trips = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                a = trip_set[j-1]
                b = trip_set[j]
                dict_adjacent_trips[str(i),str(a),str(b)] = 1
        #print(dict_adjacent_trips)

        # store the available time periods for charging before each trip of each bus to a dictionary
        dict_available_charging_time = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                trip1 = trip_set[j-1]
                trip2 = trip_set[j]
                trip1_endtime = trip_information_3line.iloc[trip1-1]["End_time"]
                trip2_starttime = trip_information_3line.iloc[trip2-1]["Start_time"]
                trip1_endterminal = trip_information_3line.iloc[trip1-1]["End_terminal"]
                trip2_startterminal = trip_information_3line.iloc[trip2-1]["Start_terminal"]
                between_distance = terminal_distance_3line.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
                between_time_need = math.ceil(between_distance/20*60)
                for l in range(int(trip1_endtime+between_time_need), int(trip2_starttime)):
                    dict_available_charging_time[str(i),str(trip2),str(int(trip2_startterminal)),str(l)] = 1
        #print(dict_available_charging_time)

        # store the available time periods for charging at the depot for each bus
        dict_available_charging_time_d = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            last_trip = trip_set[-1]
            first_trip = trip_set[0]
            last_trip_end_terminal = trip_information_3line.iloc[last_trip-1]["End_terminal"]
            last_trip_end_time = trip_information_3line.iloc[last_trip-1]["End_time"]
            first_trip_start_terminal = trip_information_3line.iloc[first_trip-1]["Start_terminal"]
            first_trip_start_time = trip_information_3line.iloc[first_trip-1]["Start_time"]
            arrive_depot_time = last_trip_end_time + math.ceil(From_trip_to_depot_time.iloc[last_trip-1][1]*60)
            leave_depot_time = first_trip_start_time - math.ceil(From_depot_to_trip_time.iloc[first_trip-1][1]*60)
            for t in range(int(arrive_depot_time),1441):
                dict_available_charging_time_d[str(i),str(t)] =1
            for t in range(1,int(leave_depot_time)):
                dict_available_charging_time_d[str(i),str(t)] =1
        #print(dict_available_charging_time_d)

        # store the start terminal of trip i to a dictionary
        dict_start_terminal = {}
        for i in range(70):
            start_terminal = trip_information_3line.iloc[i]["Start_terminal"]
            dict_start_terminal[str(i+1), str(int(start_terminal))] = 1
        #print(dict_start_terminal)

        # store the end terminal of trip i to a dictionary
        dict_end_terminal = {}
        for i in range(70):
            end_terminal = trip_information_3line.iloc[i]["End_terminal"]
            dict_end_terminal[str(i+1), str(int(end_terminal))] = 1
        #print(dict_end_terminal)

        # store the energy consumption of serving a trip of each bus to a dictionary
        dict_energy_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in trip_set:
                trip_energy_need = trip_information_3line.iloc[j-1]["Energy_need"]
                dict_energy_trip[str(i),str(j)] = trip_energy_need
        #print(dict_energy_trip)

        # store the energy consumption of serving two adjacent trips of each bus to a dictionary
        dict_energy_between_trips = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            for j in range(1, len(trip_set)):
                trip1 = trip_set[j-1]
                trip2 = trip_set[j]
                trip1_endterminal = trip_information_3line.iloc[trip1-1]["End_terminal"]
                trip2_startterminal = trip_information_3line.iloc[trip2-1]["Start_terminal"]
                between_distance = terminal_distance_3line.iloc[int(trip1_endterminal)-1][int(trip2_startterminal)]
                between_energy_need = between_distance*2.69
                dict_energy_between_trips[str(i),str(trip1),str(trip2)] = between_energy_need
        #print(dict_energy_between_trips)

        # store the energy consumption from the depot to the first trip of each bus to a dictionary
        dict_energy_depot_to_first_trip = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            first_trip = trip_set[0]
            dict_energy_depot_to_first_trip[str(i),str(first_trip)] = From_depot_to_trip_energy.iloc[first_trip-1][1]
        #print(dict_energy_depot_to_first_trip)

        # store the energy consumption from the last trip to the depot of each bus to a dictionary
        dict_energy_last_trip_to_depot = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            last_trip = trip_set[-1]
        #    print(last_trip)
            dict_energy_last_trip_to_depot[str(i),str(last_trip)] = From_trip_to_depot_energy.iloc[last_trip-1][1]
        #print(dict_energy_last_trip_to_depot)

        # store the total operating time per day per bus
        dict_operating_time = {}
        for i in range(1,fleet_size+1):
            trip_set = trip_chains[i-1]
            first_trip = trip_set[0]
            last_trip = trip_set[-1]
            time1 = From_depot_to_trip_time.iloc[first_trip-1][1]
            first_trip_start_time = trip_information_3line.iloc[first_trip-1]["Start_time"]
            last_trip_end_time = trip_information_3line.iloc[last_trip-1]["End_time"]
            time2 = last_trip_end_time - first_trip_start_time
            time3 = From_trip_to_depot_time.iloc[last_trip-1][1]
            dict_operating_time[str(i)] = time1*60 + time2 + time3*60
        #print(dict_operating_time)  

        # store whether a terminal is equipped with a charging station
        dict_term_charging = {}
        for i in range(cs):
            dict_term_charging[str(i+1)] = Charging_station[i]
        #print(dict_term_charging)

        ws = GamsWorkspace(system_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/", working_directory="/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS")
        # create new GamsDatabase instance
        db = ws.add_database()

        fleet_size_set = []
        i=1
        while i <= fleet_size:
            fleet_size_set.append(str(i))
            i += 1

        # add 1-dimensional set 'm' with explanatory text 'bus index' to the GamsDatabase
        m = db.add_set("m", 1, "bus index")
        for i in fleet_size_set:
            m.add_record(i)

        # add parameter 'batt' with domain 'm' with explanatory text 'battery capacity of bus bbb' to the GamsDatabase
        batt = db.add_parameter_dc("batt", [m], "battery capacity of bus bbb")
        for i in fleet_size_set:
            batt.add_record(i).value = chrom_batt[0][int(i)-1]

        # add parameter to define the trips that each bus serves
        bust = GamsParameter(db, "bust", 2, "whether a trip is served by a bus")
        for k, v in dict_trip_set.items():
            bust.add_record(k).value = v

        # add parameter to define the first trip of each bus
        busf = GamsParameter(db, "busf", 2, "whether a trip is the first trip of a bus")
        for k, v in dict_first_trip.items():
            busf.add_record(k).value = v

        # add parameter to define the last trip of each bus
        busl = GamsParameter(db, "busl", 2, "whether a trip is the last trip of a bus")
        for k, v in dict_last_trip.items():
            busl.add_record(k).value = v

        # add parameter to define the two adjacent trips of each bus
        busaj = GamsParameter(db, "busaj", 3, "whether two trips are adjacent trips of a bus")
        for k, v in dict_adjacent_trips.items():
            busaj.add_record(k).value = v

        # add parameter to define the available charging time periods before serving a trip of each bus
        busct = GamsParameter(db, "busct", 4, "available charging time periods before serving a trip of a bus")
        for k, v in dict_available_charging_time.items():
            busct.add_record(k).value = v

        # add parameter to define the available charging time periods at the depot for each bus
        busctd = GamsParameter(db, "busctd", 2, "available charging time periods at the depot of a bus")
        for k, v in dict_available_charging_time_d.items():
            busctd.add_record(k).value = v

        # add parameter to define the start terminal of each trip
        tripst = GamsParameter(db, "tripst", 2, "start terminal of a trip")
        for k, v in dict_start_terminal.items():
            tripst.add_record(k).value = v

        # add parameter to define the end terminal of each trip
        tripet = GamsParameter(db, "tripet", 2, "end terminal of a trip")
        for k, v in dict_end_terminal.items():
            tripet.add_record(k).value = v

        # add parameter to define the energy need for serving a trip of each bus
        busen = GamsParameter(db, "busen", 2, "energy need for serving a trip of a bus")
        for k, v in dict_energy_trip.items():
            busen.add_record(k).value = v

        # add parameter to define the energy consumption between two adjacent trips of each bus
        busea = GamsParameter(db, "busea", 3, "energy consumption between two adjacent trips of a bus")
        for k, v in dict_energy_between_trips.items():
            busea.add_record(k).value = v

        # add parameter to define the energy consumption from the depot to the first trip of each bus
        busdf = GamsParameter(db, "busdf", 2, "energy consumption from the depot to the first trip of a bus")
        for k, v in dict_energy_depot_to_first_trip.items():
            busdf.add_record(k).value = v

        # add parameter to define the energy consumption from the last trip to the depot of each bus
        busld = GamsParameter(db, "busld", 2, "energy consumption from the last trip to the depot of a bus")
        for k, v in dict_energy_last_trip_to_depot.items():
            busld.add_record(k).value = v

        # add parameter to define the operating time of each bus
        busst = GamsParameter(db, "busst", 1, "operating time of each bus")
        for k, v in dict_operating_time.items():
            busst.add_record(k).value = v

        # add parameter to define whether a terminal is equipped with a charging station
        terch = GamsParameter(db, "terch", 1, "whether a terminal is equipped with a charging station")
        for k, v in dict_term_charging.items():
            terch.add_record(k).value = v

        # export the GamsDatabase to a GDX file with name 'data.gdx' located in the 'working_directory' of the GamsWorkspace
        db.export("/Applications/GAMS30.3/GAMS Terminal.app/Contents/MacOS/data.gdx")

        # call and run gams model to calculate the fitness value of the chromosome
        gams_model = ws.add_job_from_file("EBus_scheduling_new.gms")
        gams_model.run()
        if (gams_model.out_db["modelStat"].find_record().value == 1 and gams_model.out_db["solveStat"].find_record().value == 1):
            for rec in gams_model.out_db["objvalue"]:
                fit_value = -rec.level
        else:
            fit_value = -999999999
        print(fit_value)
            
        fitness_list.append(fit_value)
        fitness_return_list.append(fit_value)

    fitness = np.array(fitness_list)
    fitness_return = np.array(fitness_return_list)
    #print(fitness_return)

    parents = {}
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num] = init_population[max_fitness_idx+1]
        fitness[max_fitness_idx] = -99999999999
    return parents,fitness_return 

###########crossover
def crossover_g(parents, offspring_size):
    num_parents = len(parents)
    offspring = {}
    for k in range(offspring_size):
        # Index of the first parent to mate.
        parent1_idx = k%num_parents
        # Index of the second parent to mate.
        parent2_idx = (k+1)%num_parents
        parent1 = parents[parent1_idx]
        parent2 = parents[parent2_idx]
        off_spring = cross_over(parent1, parent2)
        offspring[k] = off_spring
    return offspring

###########mutation
def mutation_g(offspring_crossover,mutationRate):
    num_mutation = int(mutationRate*len(offspring_crossover))
    for idx in range(num_mutation):
        mutation_idx = idx%num_mutation
        offspring_crossover[mutation_idx] = mutation(offspring_crossover[mutation_idx])
    return offspring_crossover

###########genetic algorithm
maximal_generation=150
population_size=120
parent_number=36
#maximal_generation=50
#population_size=20
#parent_number=16
mutation_rate=0.15
#number_of_variables = 3 #Battery size, charger number, charger power, charging threshold    
child_number = population_size - parent_number



In [28]:
new_population = dict_feasible_chromosome
best_outputs = []
for i in range(maximal_generation):
    print("Generation: "+str(i))
    parents,fitness_return = select_mating_pool(new_population, parent_number)
    offspring_crossover = crossover_g(parents, offspring_size=child_number)
    offspring_mutation = mutation_g(offspring_crossover, mutation_rate)
    for key,value in parents.items():
        new_population[key+1] = value
    for key,value in offspring_mutation.items():
        new_population[key+parent_number+1] = value

    print(fitness_return)
    print(np.max(fitness_return))


    max_fitness_idx = np.where(fitness_return == np.max(fitness_return))
    max_fitness_idx = max_fitness_idx[0][0]
    best_solution = (np.max(fitness_return),new_population[max_fitness_idx+1])
    best_outputs.append(best_solution)

#print(best_solution)

Generation: 0


GamsException: Error retrieving audit info

In [150]:
randrange(1,2)

1